In [1]:
!nvidia-smi

Wed Mar 20 13:57:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 21.2 MB/

In [3]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 981.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 384.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 511.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 564.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 188.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 221.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━ 85.7/121.6 MB 759.4 kB/s eta 0:00:48
ERROR: THESE PACKAGES DO NOT MATCH THE HASHES FROM THE REQUIREMENTS FILE. If you have updated the package versions, please update the hashes. Otherwise, examine the package contents carefully; someone may have tampered with them.
    nvidia-cufft-cu12==11.0.2.54 from https://files.pythonhosted.org/packages/86/94/eb540db023ce1d162e7bea9f8f5aa781d57c65aed513c33ee9a5123ead4d/nvidia_cufft_cu12-11.0.2.54-py3-none-manyl

In [4]:
import os

# Set CUDA_LAUNCH_BLOCKING environment variable
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [5]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [7]:
model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [8]:
#dowload & unzip data

#!wget https://github.com/InesFRibeiro/meta-review/raw/main/meta-review.zip
#!unzip meta-review.zip

--2024-03-20 14:32:33--  https://github.com/InesFRibeiro/meta-review/raw/main/meta-review.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/InesFRibeiro/meta-review/main/meta-review.zip [following]
--2024-03-20 14:32:34--  https://raw.githubusercontent.com/InesFRibeiro/meta-review/main/meta-review.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39860551 (38M) [application/zip]
Saving to: ‘meta-review.zip’

meta-review.zip     100%[===================>]  38.01M   184MB/s    in 0.2s    

2024-03-20 14:32:34 (184 MB/s) - ‘meta-review.zip’ saved [39860551/39860551]

Archive:  meta-review.zip
 

In [9]:
# Load the data
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
validation_data = pd.read_csv("validation.csv")

In [10]:
train_data['id'] = train_data.index.values + 1
train_data = train_data[['id', 'Input', 'Output']]

test_data = pd.read_csv("test.csv")
test_data['id'] = test_data.index.values + 1
test_data = test_data[['id', 'Input', 'Output']]

validation_data = pd.read_csv("validation.csv")
validation_data['id'] = validation_data.index.values + 1
validation_data = validation_data[['id', 'Input', 'Output']]


train_data

,id,Input,Output
0,1,"In this paper, the author investigates how to ...",This paper studies how to learn dexterous mani...
1,2,**Summary of contributions:** This paper propo...,This paper proposed a new family of losses for...
2,3,This paper addresses the problem of MoE routin...,Mixture-of-Expert (MoE) models have demonstrat...
3,4,This paper discusses applications of variants ...,"In this work, the authors conduct experiments ..."
4,5,The authors introduce the problem of telegraph...,This paper presents methods for telegraphic su...
...,...,...,...
7687,7688,The paper proposes a method to use CLIP for or...,The paper proposes a language-powered model fo...
7688,7689,This work concerns the asymptotic behaviours o...,All reviewers are in agreement that the main f...
7689,7690,The paper surveys recent offline RL algorithms...,The main strengths of this paper are that (1) ...
7690,7691,This paper proposes an algorithm for off-polic...,The reviewer concerns generally centered aroun...


In [11]:
#split_lengths = [len(dataset_samsum[split])for split in dataset_samsum]

#print(f"Split lengths: {split_lengths}")
print(f"Features: {train_data.columns.values}")

print("\nTex:")
print(train_data.iloc[:1,1:2].values[0][0])

print("\nSummary:")
print(train_data.iloc[:1,2:].values[0][0])

Features: ['id' 'Input' 'Output']

Tex:
In this paper, the author investigates how to utilize large-scale human video to train dexterous robot manipulation skills. To leverage the information from the Internet videos, the author proposes a handful of techniques to pre-process the video data to extract the action information. Then the network is trained on the extracted hand data and deployed to the real robot with some human demonstration collected by teleoperation for fine-tuning. Experiments show that the proposed pipeline can solve multiple manipulation tasks.  **Strength**  - The direction explored in this paper is important. Utilizing the internet video data for robot learning is well motivated. Especially considering the similarity between human and multi-finger hands, this direction looks very promising.   - The authors perform experiments with multiple real-world tasks with pick and place, pushing, and rotating objects.  **Weakness**  - Although the objective of this paper is v

In [12]:
# Define the dataset
from datasets import DatasetDict, Dataset

dataset_meta_review = DatasetDict({
    "train": Dataset.from_pandas(train_data),
    "test": Dataset.from_pandas(test_data),
    "validation": Dataset.from_pandas(validation_data)
})

# Filter out any rows that contain missing values
dataset_meta_review = dataset_meta_review.filter(lambda x: x['Input'] is not None and x['Output'] is not None)

Filter:   0%|          | 0/7692 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1649 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1648 [00:00<?, ? examples/s]

In [13]:
def convert_examples_to_features(example_batch):
    input_texts = example_batch['Input']
    target_texts = example_batch['Output']

    # Concatenate the input and target texts into a single list of strings
    input_target_texts = [input_text + target_text for input_text, target_text in zip(input_texts, target_texts) if input_text is not None and target_text is not None]

    print("Input texts:", input_texts)
    print("Target texts:", target_texts)
    print("Input-target texts:", input_target_texts)

    # Encode the input and target texts
    encodings = tokenizer(input_target_texts, max_length=10000, truncation=True, padding='max_length', return_tensors='pt')

    print("Encodings:", encodings)

    # Extract the input and target encodings
    input_encodings = encodings['input_ids'][:, :len(input_texts[0])]
    target_encodings = encodings['input_ids'][:, len(input_texts[0]):]

    return {
        'input_ids': input_encodings,
        'attention_mask': encodings['attention_mask'],
        'labels': target_encodings
    }

# Map the data to features
dataset_meta_review_pt = dataset_meta_review.map(convert_examples_to_features, batched=True)

Map:   0%|          | 0/7680 [00:00<?, ? examples/s]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Encodings: {'input_ids': tensor([[  222,   136,   800,  ...,     0,     0,     0],
        [12937, 22071,   113,  ...,     0,     0,     0],
        [  182,   800,  4183,  ...,     0,     0,     0],
        ...,
        [  182,   800,  1683,  ...,     0,     0,     0],
        [  139,  3802,  4094,  ...,     0,     0,     0],
        [  182,   800,  1683,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Encodings: {'input_ids': tensor([[56304, 78575, 78575,  ...,     0,     0,     0],
        [  182,   800,   117,  ...,     0,     0,     0],
        [ 9863, 41087,   143,  ...,     0,     0,     0],
        ...,
        [  182,   800,  9905,  ...,     0,     0,     0],
        [  182,   800, 19390,  ...,     0,     0,     0],
        [  110, 29318,   151,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


...

Map:   0%|          | 0/1645 [00:00<?, ? examples/s]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Encodings: {'input_ids': tensor([[  182,   800,  3702,  ...,     0,     0,     0],
        [  139,   800,  4795,  ...,     0,     0,     0],
        [  139,  6733,   939,  ...,     0,     0,     0],
        ...,
        [  182,   201,  9905,  ...,     0,     0,     0],
        [15567,   113,   933,  ...,     0,     0,     0],
        [  143, 63419,   122,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Encodings: {'input_ids': tensor([[  182,   800,   692,  ...,     0,     0,     0],
        [  139,  3802,  1697,  ...,     0,     0,     0],
        [  182,   800, 19390,  ...,     0,     0,     0],
        ...,
        [  139,   800,  2962,  ...,     0,     0,     0],
        [12937,   251,  3598,  ...,     0,     0,     0],
        [  139,   800, 19390,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [14]:
# Define the dataset
from datasets import DatasetDict, Dataset

# Filter out any rows that contain missing values
train_data = train_data.dropna(subset=['Input', 'Output'])
test_data = test_data.dropna(subset=['Input', 'Output'])
validation_data = validation_data.dropna(subset=['Input', 'Output'])

# Define the dataset
dataset_meta_review = DatasetDict({
    "train": Dataset.from_pandas(train_data),
    "test": Dataset.from_pandas(test_data),
    "validation": Dataset.from_pandas(validation_data)
})

# Tokenize input and output sequences
def tokenize_data(example):
    inputs = tokenizer(example['Input'], truncation=True, padding='max_length', max_length=10000)
    outputs = tokenizer(example['Output'], truncation=True, padding='max_length', max_length=10000)

    return {'input_ids': inputs['input_ids'],
            'attention_mask': inputs['attention_mask'],
            'labels': outputs['input_ids']}

dataset_meta_review = dataset_meta_review.map(tokenize_data, batched=True)


Map:   0%|          | 0/7680 [00:00<?, ? examples/s]

Map:   0%|          | 0/1645 [00:00<?, ? examples/s]

Map:   0%|          | 0/1645 [00:00<?, ? examples/s]

In [15]:
# Training
from transformers import DataCollatorForSeq2Seq, TrainingArguments, Trainer

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

trainer_args = TrainingArguments(
    output_dir='pegasus-meta-review',
    num_train_epochs=3,
    warmup_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    logging_steps=10,
    evaluation_strategy='steps',
    eval_steps=500,
    save_steps=1e6,
    gradient_accumulation_steps=16
)

trainer = Trainer(
    model=model_pegasus,
    args=trainer_args,
    tokenizer=tokenizer,
    data_collator=seq2seq_data_collator,
    train_dataset=dataset_meta_review["train"],
    eval_dataset=dataset_meta_review["validation"]
)

# Start training
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=80000,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=5000)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score


In [ ]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')

In [ ]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="Input",
                               column_summary="Output"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=80000,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=5000)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score


In [ ]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')


/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
score = calculate_metric_on_test_ds(
    dataset_meta_review['test'], rouge_metric, model_pegasus, tokenizer, batch_size=2, column_text='Input', column_summary='Output'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )

  0%|          | 0/823 [00:00<?, ?it/s]


RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Calculate metric on test dataset
score = calculate_metric_on_test_ds(
    dataset_meta_review['test'], rouge_metric, model_pegasus, tokenizer,
    batch_size=2, device=device, column_text='Input', column_summary='Output'
)

# Extract ROUGE scores
rouge_dict = {rn: score[rn].mid.fmeasure for rn in rouge_names}

# Create DataFrame
df_rouge = pd.DataFrame(rouge_dict, index=['pegasus'])

df_rouge


  0%|          | 0/823 [00:00<?, ?it/s]


RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
## Save model
model_pegasus.save_pretrained("pegasus-review-model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
## Save tokenizer
tokenizer.save_pretrained("review_tokenizer")

('review_tokenizer/tokenizer_config.json',
 'review_tokenizer/special_tokens_map.json',
 'review_tokenizer/spiece.model',
 'review_tokenizer/added_tokens.json',
 'review_tokenizer/tokenizer.json')

In [ ]:
#Load
tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")

In [ ]:
#Prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 10000}



sample_text = dataset_samsum["test"][0]["Input"]

reference = dataset_samsum["test"][0]["Output"]

pipe = pipeline("summarization", model="pegasus-samsum-model",tokenizer=tokenizer)

##
print("Text:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Your max_length is set to 10000, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:


KeyboardInterrupt: 